<a href="https://colab.research.google.com/github/kimheeseo/pytorch-RNN/blob/main/RNN_%EC%97%B0%EC%8A%B5_2021_12_04(2).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
#출처 : https://justkode.kr/deep-learning/pytorch-rnn
sentences = ["i like dog", "i love coffee", "i hate milk", "you like cat", "you love milk", "you hate coffee"]
dtype = torch.float

"""
Word Processing
"""
word_list = list(set(" ".join(sentences)))
print('word_list값',word_list)


word_list값 ['l', 'h', 'g', 'k', 'a', 'c', 'v', 'i', 'e', 't', 'f', 'o', 'u', ' ', 'y', 'd', 'm']


In [20]:
word_list = list(set(" ".join(sentences).split()))
print(word_list)
word_dict = {w: i for i, w in enumerate(word_list)}
print('word_dict값',word_dict)
number_dict = {i: w for i, w in enumerate(word_list)}
print('number_dict값',number_dict)
n_class = len(word_dict)
print('n_class값',n_class)

['cat', 'like', 'you', 'coffee', 'milk', 'i', 'hate', 'love', 'dog']
word_dict값 {'cat': 0, 'like': 1, 'you': 2, 'coffee': 3, 'milk': 4, 'i': 5, 'hate': 6, 'love': 7, 'dog': 8}
number_dict값 {0: 'cat', 1: 'like', 2: 'you', 3: 'coffee', 4: 'milk', 5: 'i', 6: 'hate', 7: 'love', 8: 'dog'}
n_class값 9


In [21]:
"""
TextRNN Parameter
"""
batch_size = len(sentences)
print('batch_size값',batch_size)
n_step = 2  # 학습 하려고 하는 문장의 길이 - 1
n_hidden = 5  # 은닉층 사이즈
print('sentences값',sentences)

inputtest_batch=[]
targettest_batch=[]
i=0
#다음 for문은 make_batch문의 동작을 보기위해서, 나눠서 표현방법을 본다.
for sen in sentences:
  print(i)
  i=i+1
  word=sen.split()
  print('word값',word)
  input=[word_dict[n] for n in word[:]]
  print('input값',input)
  target=word_dict[word[-1]]
  print('word[-1]값',word[-1])
  #print('np.eye(n_class)값',np.eye(n_class))
  #print('np.eye(n_class)[input]',np.eye(n_class)[input])
  inputtest_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
  #print('inputtest_batch값',inputtest_batch)
  print('target값',target)
  targettest_batch.append(target)

print('inputtest_batch값',inputtest_batch)
print('targettest_batch값',targettest_batch)

#split에 의해 1번째 for문 동작 : 'I like dog'이 sen값이 된다. 그리고 "word=sen.split()"에 의해서
#word값은 ['I','like']가 되고, input값은 [5,1]이 된다. 우선 2개의 값이 나오는 이유는 [:-1]에 의해서 그런것이고,
#만일 [:]이면, ['I','like','dog']가 되고 input값은 [5,1,8]이 되고, 이때 숫자는 앞서 word_dict을 통해 설정한
#I는 5, like는 1, dog는 8을 나타내기 때문에, 이와 같은 결과값이 나온다.

batch_size값 6
sentences값 ['i like dog', 'i love coffee', 'i hate milk', 'you like cat', 'you love milk', 'you hate coffee']
0
word값 ['i', 'like', 'dog']
input값 [5, 1, 8]
word[-1]값 dog
target값 8
1
word값 ['i', 'love', 'coffee']
input값 [5, 7, 3]
word[-1]값 coffee
target값 3
2
word값 ['i', 'hate', 'milk']
input값 [5, 6, 4]
word[-1]값 milk
target값 4
3
word값 ['you', 'like', 'cat']
input값 [2, 1, 0]
word[-1]값 cat
target값 0
4
word값 ['you', 'love', 'milk']
input값 [2, 7, 4]
word[-1]값 milk
target값 4
5
word값 ['you', 'hate', 'coffee']
input값 [2, 6, 3]
word[-1]값 coffee
target값 3
inputtest_batch값 [array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 0., 1.]]), array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0., 0., 0., 0., 0.]]), array([[0., 0., 0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 0., 1., 0., 0.],
       [0., 0., 0., 0., 1., 0., 0., 0., 0.]]), array

In [22]:
def make_batch(sentences):
  input_batch = []
  target_batch = []

  for sen in sentences:
    word = sen.split()
    input = [word_dict[n] for n in word[:-1]]
    target = word_dict[word[-1]]

    input_batch.append(np.eye(n_class)[input])  # One-Hot Encoding
    target_batch.append(target)
  
  return input_batch, target_batch  

In [23]:
input_batch, target_batch = make_batch(sentences)
input_batch = torch.tensor(input_batch, dtype=torch.float32, requires_grad=True)
target_batch = torch.tensor(target_batch, dtype=torch.int64)
print('input_batch값',input_batch)
print('target_batch값',target_batch)

input_batch값 tensor([[[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 1., 0.]],

        [[0., 0., 1., 0., 0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0., 0., 1., 0., 0.]]], requires_grad=True)
target_batch값 tensor([8, 3, 4, 0, 4, 3])


In [27]:
"""
TextRNN
"""
class TextRNN(nn.Module):
  def __init__(self):
    super(TextRNN, self).__init__()

    self.rnn = nn.RNN(input_size=n_class, hidden_size=n_hidden, dropout=0.3)
    self.W = nn.Parameter(torch.randn([n_hidden, n_class]).type(dtype))
    self.b = nn.Parameter(torch.randn([n_class]).type(dtype))
    self.Softmax = nn.Softmax(dim=1)

  def forward(self, hidden, X):
    X = X.transpose(0, 1)
    outputs, hidden = self.rnn(X, hidden)
    #print('outputs값', outputs)
    #print('hidden값', hidden)
    outputs = outputs[-1]  # 최종 예측 Hidden Layer
    model = torch.mm(outputs, self.W) + self.b  # 최종 예측 최종 출력 층
    print('model값',model)
    return model
	
"""
Training
"""
model = TextRNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

for epoch in range(500):
  hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
  output = model(hidden, input_batch)
  loss = criterion(output, target_batch)

  if (epoch + 1) % 100 == 0:
    print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
  
  optimizer.zero_grad()
  loss.backward()
  optimizer.step()

input = [sen.split()[:2] for sen in sentences]
print('input값',input)
print('output값',output)

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/rnn.py:65: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.3 and num_layers=1
  "num_layers={}".format(dropout, num_layers))


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
        [ 0.7929,  0.0821,  0.4167,  3.2452,  1.7360, -4.6136, -2.1668, -3.3562,
         -0.0292]], grad_fn=<AddBackward0>)
model값 tensor([[ 0.5674, -2.3040, -2.1889,  0.7648,  1.1734, -6.7416, -2.6442, -5.2272,
          1.9928],
        [-2.3660, -1.7040, -2.6185,  1.8609,  2.5339, -5.4748, -2.4315, -3.4351,
          1.0302],
        [-1.8287, -1.8466, -2.1007,  2.0348,  2.3407, -5.8262, -2.4191, -3.6862,
          1.3634],
        [ 3.7253, -1.1918, -1.1437,  0.6316,  0.0295, -6.1793, -2.5361, -5.9212,
          1.1903],
        [-0.8772, -0.2007, -1.7708,  2.0993,  2.1855, -4.2635, -2.3703, -3.1566,
         -0.3880],
        [ 0.7854,  0.1065,  0.4458,  3.2799,  1.7531, -4.5916, -2.1764, -3.3433,
         -0.0631]], grad_fn=<AddBackward0>)
model값 tensor([[ 0.5711, -2.3140, -2.1902,  0.7539,  1.1704, -6.7571, -2.6670, -5.2500,
          2.0133],
        [-2.3906, -1.7093, -2.6306,  1.8748,  2.5476, -5.4681, -2.4465, -3.4428,
          1.0250],


In [25]:
hidden = torch.zeros(1, batch_size, n_hidden, requires_grad=True)
print('hidden값',hidden)
#data.max(1,keepdim=True)[1] 관련 자료 : https://blog.naver.com/PostView.nhn?blogId=na_young_1124&logNo=221455757665&from=search&redirect=Log&widgetTypeCall=true&directAccess=false
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
predict2 = model(hidden, input_batch).data.max(1, keepdim=True)[0]
predict3 = model(hidden, input_batch).data.max(1, keepdim=True)

print('predict3값',predict3)
print('predict2값',predict2)
print('predict값',predict)
print([sen.split()[:2] for sen in sentences], '->', [number_dict[n.item()] for n in predict.squeeze()])

hidden값 tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], requires_grad=True)
model값 tensor([[ 1.1637e+00, -1.6393e+00, -5.3479e-01,  3.9796e+00, -3.5225e+00,
         -1.9087e-03, -2.4379e+00,  1.8274e+00,  8.7745e+00],
        [-3.8508e+00, -2.5374e+00,  6.6924e-01,  7.1172e+00,  1.7292e+00,
         -4.3164e+00, -3.4031e+00, -7.0395e-01,  1.9948e+00],
        [-4.0444e+00, -1.5563e+00, -1.7590e+00,  2.4197e+00,  7.8389e+00,
         -2.4448e+00, -3.1883e+00, -6.4439e-02, -3.1048e+00],
        [ 6.4498e+00, -3.2275e+00, -2.0795e+00, -6.2131e-01, -8.9005e-02,
         -8.9779e-01, -3.7208e+00, -5.5229e-01,  2.5984e-01],
        [ 1.4398e-01, -4.1415e+00, -1.0574e+00,  9.3144e-01,  6.1009e+00,
         -2.1195e+00, -4.2889e+00, -3.3292e+00, -5.7157e+00],
        [ 1.5945e+00, -3.4162e+00, -3.2029e+00,  7.4629e+00,  2.1562e+00,
         -6.5602e+00, -

In [29]:
predict = model(hidden, input_batch).data.max(1, keepdim=True)[1]
print('hidden값',hidden)
print('input_batch값',input_batch)
print(number_dict[n.item()] for n in predict.squeeze())
print(predict.squeeze())

model값 tensor([[ 0.4413, -2.7469, -1.5746,  0.1759, -1.0650, -8.2182, -3.9098, -7.5187,
          5.7391],
        [-7.5321, -4.0994, -3.9688,  6.8452,  2.5983, -6.8955, -3.2608, -4.9941,
          1.9484],
        [-2.6155, -1.4729, -2.8138,  0.4233,  5.5265, -4.3990, -3.6506, -4.3820,
         -2.4575],
        [ 7.3637,  0.7935, -0.1137, -0.3993,  0.9536, -2.7258, -1.1387, -4.8616,
         -0.4302],
        [-3.4766, -1.4548, -3.7162,  2.3188,  6.4622, -3.1802, -2.7187, -3.4474,
         -3.9108],
        [ 2.3739,  0.8429,  0.9990,  8.4546,  3.5676, -1.7747, -0.9733, -2.0657,
         -4.9190]], grad_fn=<AddBackward0>)
hidden값 tensor([[[0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.],
         [0., 0., 0., 0., 0.]]], requires_grad=True)
input_batch값 tensor([[[0., 0., 0., 0., 0., 1., 0., 0., 0.],
         [0., 1., 0., 0., 0., 0., 0., 0., 0.]],

        [[0., 0., 0., 0., 0., 1., 0., 0., 0